In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
from torch import nn
from torch.nn.utils import prune
import numpy as np
import sys
import matplotlib.pyplot as plt
import math
import random
import sklearn.metrics as perf
import os
import cv2
import time
import torch_pruning as tp
import csv

from models.models import MTLClassifier, AgeRegressor, GenderClassifier, EthnicityClassifier
from utils.data import FacesDataset, data_transform
from utils.training import train_mtl_model, train_age_model, train_gender_model, train_ethnicity_model, train_mtl_model_individual
from utils.evaluation import run_evaluation, show_example_predictions
from utils.pruning import prune_model, prune_other_tasks, get_f1_and_lat

### Load and Prepare Data

In [2]:
### Load in the data
folder = 'UTKFace'
transform = data_transform()
dataset = FacesDataset(folder=folder, transform=transform)

In [3]:
### Set up train and val datasets and loaders
train_len = int(len(dataset)*0.8)
val_len = len(dataset) - train_len
train_dataset, val_dataset = random_split(dataset, [train_len, val_len], torch.Generator().manual_seed(8))

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=16, shuffle=False)

In [4]:
### Get small subset fpr train_loader and val_loader (for testing and debugging)
# train_indices = torch.randperm(len(train_dataset))[:100]
# val_indices = torch.randperm(len(val_dataset))[:100]
# train_subset = torch.utils.data.Subset(train_dataset, train_indices)
# val_subset = torch.utils.data.Subset(val_dataset, val_indices)
# train_loader = DataLoader(dataset=train_subset, batch_size=16, shuffle=True)
# val_loader = DataLoader(dataset=val_subset, batch_size=16, shuffle=True)

### MTL Model Variants

#### Define pruned training function

In [5]:
def pruned_mtl_training(task, prune_pct, num_epochs,
                        train_loader=train_loader, val_loader=val_loader,
                        val_dataset=None):
    
    ### Set up model, loss, and optimizer
    if task.upper()=='ALL':
        tasks = ['age', 'gender', 'ethnicity']
        model = MTLClassifier()
        model = model.cuda()
        age_criterion = nn.MSELoss()
        gender_criterion = nn.CrossEntropyLoss()
        ethnicity_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        age_coeff = 0.004
        gender_coeff = 2
        ethnicity_coeff = 1
        
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_MTLModel.pth'
        train_mtl_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                        train_loader=train_loader, val_loader=val_loader,
                        age_criterion=age_criterion, gender_criterion=gender_criterion, ethnicity_criterion=ethnicity_criterion,
                        age_coeff=age_coeff, gender_coeff=gender_coeff, ethnicity_coeff=ethnicity_coeff, save=True,
                        save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_model(model, PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_MTLModel.pth'
        train_mtl_model(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                        train_loader=train_loader, val_loader=val_loader,
                        age_criterion=age_criterion, gender_criterion=gender_criterion, ethnicity_criterion=ethnicity_criterion,
                        age_coeff=age_coeff, gender_coeff=gender_coeff, ethnicity_coeff=ethnicity_coeff, save=True,
                        save_name=save_initial)
        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                   eval_dataset=val_dataset,
                                                   eval_dataloader=val_loader,
                                                   tasks=tasks,
                                                   mtl_model=True)
        
        # Save model with score and latency information in the model name
        ager2 = scores['age'][1]
        genderf1 = scores['gender'][0]
        ethnicityf1 = scores['ethnicity'][0]
        row = [task.upper(), prune_pct, mean_lat, std_lat, ager2, genderf1, ethnicityf1]
        with open('models/model_variants/model_score_lookup_multitask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_MTLModel.pth'
        torch.save(model, f"models/{new_name}")
        
        
    elif task.upper()=='GENDER':
        tasks = ['gender']
        model = MTLClassifier()
        model = model.cuda()
        age_criterion = nn.MSELoss()
        gender_criterion = nn.CrossEntropyLoss()
        ethnicity_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        age_coeff = 0.004
        gender_coeff = 2
        ethnicity_coeff = 1
        
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_MTLModel.pth'
        train_mtl_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                        train_loader=train_loader, val_loader=val_loader,
                        age_criterion=age_criterion, gender_criterion=gender_criterion, ethnicity_criterion=ethnicity_criterion,
                        age_coeff=age_coeff, gender_coeff=gender_coeff, ethnicity_coeff=ethnicity_coeff, save=True,
                        save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_other_tasks(model, task1='age', task2='ethnicity', PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_MTLModel.pth'
        train_mtl_model_individual(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                              train_loader=train_loader, val_loader=val_loader,
                              gender_criterion=gender_criterion, age_coeff=1.0, gender_coeff=1.0, ethnicity_coeff=1.0,
                              save=True, save_name=save_initial, isGender=True)
        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                   eval_dataset=val_dataset,
                                                   eval_dataloader=val_loader,
                                                   tasks=tasks,
                                                   mtl_model=True)
        
        # Save model with score and latency information in the model name
        genderf1 = scores['gender'][0]
        row = [task.upper(), prune_pct, mean_lat, std_lat, 0.0, genderf1, 0.0]
        with open('models/model_variants/model_score_lookup_multitask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_MTLModel.pth'
        torch.save(model, f"models/{new_name}")
        
    elif task.upper()=='ETHNICITY':
        tasks = ['ethnicity']
        model = MTLClassifier()
        model = model.cuda()
        age_criterion = nn.MSELoss()
        gender_criterion = nn.CrossEntropyLoss()
        ethnicity_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        age_coeff = 0.004
        gender_coeff = 2
        ethnicity_coeff = 1
        
        
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_MTLModel.pth'
        train_mtl_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                        train_loader=train_loader, val_loader=val_loader,
                        age_criterion=age_criterion, gender_criterion=gender_criterion, ethnicity_criterion=ethnicity_criterion,
                        age_coeff=age_coeff, gender_coeff=gender_coeff, ethnicity_coeff=ethnicity_coeff, save=True,
                        save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_other_tasks(model, task1='age', task2='gender', PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_MTLModel.pth'
        train_mtl_model_individual(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                              train_loader=train_loader, val_loader=val_loader,
                              ethnicity_criterion=ethnicity_criterion, age_coeff=1.0, gender_coeff=1.0, ethnicity_coeff=1.0,
                              save=True, save_name=save_initial, isethnicity=True)
        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                       eval_dataset=val_dataset,
                                                       eval_dataloader=val_loader,
                                                       tasks=tasks,
                                                       mtl_model=True)
        
        # Save model with score and latency information in the model name
        ethnicityf1 = scores['ethnicity'][0]
        row = [task.upper(), prune_pct, mean_lat, std_lat, 0.0, 0.0, ethnicityf1]
        with open('models/model_variants/model_score_lookup_multitask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_MTLModel.pth'
        torch.save(model, f"models/{new_name}")
        
    elif task.upper()=='AGE':
        tasks = ['age']
        model = MTLClassifier()
        model = model.cuda()
        age_criterion = nn.MSELoss()
        gender_criterion = nn.CrossEntropyLoss()
        ethnicity_criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Set up and run model training
        age_coeff = 0.004
        gender_coeff = 2
        ethnicity_coeff = 1
        
        
        # Train initial model
        print('---------------- Train Initial Model ----------------')
        save_no_prune = f'model_variants/{task.lower()}_p-0_lat-init_f1-init_MTLModel.pth'
        train_mtl_model(num_epochs=num_epochs, model=model, optimizer=optimizer,
                        train_loader=train_loader, val_loader=val_loader,
                        age_criterion=age_criterion, gender_criterion=gender_criterion, ethnicity_criterion=ethnicity_criterion,
                        age_coeff=age_coeff, gender_coeff=gender_coeff, ethnicity_coeff=ethnicity_coeff, save=True,
                        save_name=save_no_prune)
        
        # Do pruning
        model = torch.load(f'models/{save_no_prune}')
        pruned_model = prune_other_tasks(model, task1='gender', task2='ethnicity', PRUNING_PERCENT=prune_pct)
        pruned_optimizer = torch.optim.Adam(pruned_model.parameters())
        
        # Fine-tune model
        print('-------------- Fine-tuning Pruned Model -------------')
        save_initial = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_lat-init_f1-init_MTLModel.pth'
        train_mtl_model_individual(num_epochs=num_epochs, model=pruned_model, optimizer=pruned_optimizer,
                              train_loader=train_loader, val_loader=val_loader,
                              age_criterion=age_criterion, age_coeff=1.0, gender_coeff=1.0, ethnicity_coeff=1.0,
                              save=True, save_name=save_initial, isAge=True)

        
        # Test latency and accuracy (F1) and save model variant (and update lookup file)
        [scores, [mean_lat, std_lat]] = get_f1_and_lat(model_path=save_initial,
                                                       eval_dataset=val_dataset,
                                                       eval_dataloader=val_loader,
                                                       tasks=tasks,
                                                       mtl_model=True)
        
        # Save model with score and latency information in the model name
        ager2 = scores['age'][1]
        row = [task.upper(), prune_pct, mean_lat, std_lat, ager2, 0.0, 0.0]
        with open('models/model_variants/model_score_lookup_multitask.tsv', 'a', newline='') as f:
            writer = csv.writer(f, delimiter='\t')
            writer.writerow(row)
            
        new_name = f'model_variants/{task.lower()}_p-{int(prune_pct*100)}_MTLModel.pth'
        torch.save(model, f"models/{new_name}")
        
    else:
        print('Invalid task was specified.')
        
        
    # Delete intermediate model files
    os.remove(f'models/{save_initial}')
    if save_initial != save_no_prune:
        os.remove(f'models/{save_no_prune}')
        
    return scores, mean_lat, std_lat
        


#### 0% to 90% Pruning

In [6]:
task = 'all'
num_epochs = 10
with open('models/model_variants/model_score_lookup_multitask.tsv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(['Task', 'prune_pct', 'mean_latency', 'std_latency', 'age_r2', 'gender_f1', 'ethnicity_f1'])
            
for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.11857, train loss: 0.15138
Epoch 0, age val loss: 0.03198, gender val loss: 0.03448, ethnicity val loss: 0.05211
Epoch 1, val loss: 0.11857 -> 0.10652, train loss: 0.10865
Epoch 1, age val loss: 0.02263, gender val loss: 0.03321, ethnicity val loss: 0.05068
Epoch 2, val loss: 0.10652 -> 0.08065, train loss: 0.09472
Epoch 2, age val loss: 0.01876, gender val loss: 0.02532, ethnicity val loss: 0.03656
Epoch 3, val loss: 0.08939, train loss: 0.08537
Epoch 3, age val loss: 0.02293, gender val loss: 0.02899, ethnicity val loss: 0.03748
Epoch 4, val loss: 0.08065 -> 0.07744, train loss: 0.07740
Epoch 4, age val loss: 0.01909, gender val loss: 0.02251, ethnicity val loss: 0.03585
Epoch 5, val loss: 0.09346, train loss: 0.06696
Epoch 5, age val loss: 0.01788, gender val loss: 0.04063, ethnicity val loss: 0.03495
Epoch 6, val loss: 0.082

Epoch 6, val loss: 0.09531, train loss: 0.03676
Epoch 6, age val loss: 0.01670, gender val loss: 0.03816, ethnicity val loss: 0.04045
Epoch 7, val loss: 0.09898, train loss: 0.03434
Epoch 7, age val loss: 0.02518, gender val loss: 0.03403, ethnicity val loss: 0.03977
Epoch 8, val loss: 0.09250, train loss: 0.02946
Epoch 8, age val loss: 0.01725, gender val loss: 0.03706, ethnicity val loss: 0.03820
Epoch 9, val loss: 0.10422, train loss: 0.02647
Epoch 9, age val loss: 0.01658, gender val loss: 0.04419, ethnicity val loss: 0.04346

--------------------------------- Prune 30 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.19456, train loss: 0.15184
Epoch 0, age val loss: 0.03925, gender val loss: 0.03873, ethnicity val loss: 0.11658
Epoch 1, val loss: 0.19456 -> 0.10376, train loss: 0.11116
Epoch 1, age val loss: 0.02995, gender val loss: 0.03125, ethnicity val loss: 0.04256
Epoch 2, val loss: 0.10376 -> 0.08608, train

Epoch 2, val loss: 0.09350 -> 0.09153, train loss: 0.08101
Epoch 2, age val loss: 0.01934, gender val loss: 0.02868, ethnicity val loss: 0.04351
Epoch 3, val loss: 0.09153 -> 0.08648, train loss: 0.07145
Epoch 3, age val loss: 0.02036, gender val loss: 0.02852, ethnicity val loss: 0.03760
Epoch 4, val loss: 0.08954, train loss: 0.06376
Epoch 4, age val loss: 0.02264, gender val loss: 0.02923, ethnicity val loss: 0.03766
Epoch 5, val loss: 0.09546, train loss: 0.05744
Epoch 5, age val loss: 0.02004, gender val loss: 0.03401, ethnicity val loss: 0.04141
Epoch 6, val loss: 0.08959, train loss: 0.05303
Epoch 6, age val loss: 0.01844, gender val loss: 0.03175, ethnicity val loss: 0.03939
Epoch 7, val loss: 0.09049, train loss: 0.04715
Epoch 7, age val loss: 0.02058, gender val loss: 0.03104, ethnicity val loss: 0.03886
Epoch 8, val loss: 0.09531, train loss: 0.04467
Epoch 8, age val loss: 0.01997, gender val loss: 0.03379, ethnicity val loss: 0.04154
Epoch 9, val loss: 0.09604, train loss: 

Epoch 8, val loss: 0.07882, train loss: 0.04764
Epoch 8, age val loss: 0.01469, gender val loss: 0.02910, ethnicity val loss: 0.03502
Epoch 9, val loss: 0.08591, train loss: 0.04023
Epoch 9, age val loss: 0.01917, gender val loss: 0.03125, ethnicity val loss: 0.03549
-------------- Fine-tuning Pruned Model -------------
Epoch 0, val loss: inf -> 0.21647, train loss: 0.24011
Epoch 0, age val loss: 0.08101, gender val loss: 0.05559, ethnicity val loss: 0.07988
Epoch 1, val loss: 0.21647 -> 0.17415, train loss: 0.17871
Epoch 1, age val loss: 0.05215, gender val loss: 0.04971, ethnicity val loss: 0.07229
Epoch 2, val loss: 0.17415 -> 0.14186, train loss: 0.15802
Epoch 2, age val loss: 0.03645, gender val loss: 0.04070, ethnicity val loss: 0.06471
Epoch 3, val loss: 0.14186 -> 0.13765, train loss: 0.14756
Epoch 3, age val loss: 0.03478, gender val loss: 0.03921, ethnicity val loss: 0.06365
Epoch 4, val loss: 0.13947, train loss: 0.13988
Epoch 4, age val loss: 0.03857, gender val loss: 0.037

Notes: pruning the linear task-specific layers was sometimes causing an index-out-of-bounds error, which I think is due to it pruning off the final layer. As a fix, trying ignoring the last layer when pruning the linear layers. Note: once I edited the linear layer pruning to not prune the last layer, this problem was resolved.

However, pruning the linear layer caused more drastic performance decreases, so changed it to only prune the convolutional layers.

Note: pruning causes poor performance. Things to try: reset optimizer before pruning. This worked

Notes about this notebook. The MTL notebook generates models by first training the multitask model, pruning it, and finetuning it either for MTL or for specific tasks. The other notebook, SingleTask, does not use the MTL model or objtective at all.

#### Age

In [7]:
task = 'age'
num_epochs = 10
            
for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.11472, train loss: 0.15330
Epoch 0, age val loss: 0.03156, gender val loss: 0.03202, ethnicity val loss: 0.05115
Epoch 1, val loss: 0.11472 -> 0.10317, train loss: 0.10917
Epoch 1, age val loss: 0.02636, gender val loss: 0.02665, ethnicity val loss: 0.05016
Epoch 2, val loss: 0.10395, train loss: 0.09296
Epoch 2, age val loss: 0.02080, gender val loss: 0.04370, ethnicity val loss: 0.03945
Epoch 3, val loss: 0.10317 -> 0.08294, train loss: 0.08487
Epoch 3, age val loss: 0.02072, gender val loss: 0.02858, ethnicity val loss: 0.03364
Epoch 4, val loss: 0.08895, train loss: 0.07598
Epoch 4, age val loss: 0.02159, gender val loss: 0.02768, ethnicity val loss: 0.03968
Epoch 5, val loss: 0.08294 -> 0.08098, train loss: 0.06828
Epoch 5, age val loss: 0.02266, gender val loss: 0.02421, ethnicity val loss: 0.03411
Epoch 6, val loss: 0.084

Epoch 6, val loss: 4.23794, train loss: 2.03706
Epoch 7, val loss: 4.11519, train loss: 1.83419
Epoch 8, val loss: 3.99630, train loss: 1.74728
Epoch 9, val loss: 3.98456 -> 3.81964, train loss: 1.54445

--------------------------------- Prune 40 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.14675, train loss: 0.16017
Epoch 0, age val loss: 0.05969, gender val loss: 0.03599, ethnicity val loss: 0.05107
Epoch 1, val loss: 0.14675 -> 0.10279, train loss: 0.11397
Epoch 1, age val loss: 0.02309, gender val loss: 0.03684, ethnicity val loss: 0.04286
Epoch 2, val loss: 0.10279 -> 0.09403, train loss: 0.09687
Epoch 2, age val loss: 0.02410, gender val loss: 0.02843, ethnicity val loss: 0.04150
Epoch 3, val loss: 0.09403 -> 0.08947, train loss: 0.08653
Epoch 3, age val loss: 0.02377, gender val loss: 0.02895, ethnicity val loss: 0.03676
Epoch 4, val loss: 0.08947 -> 0.07953, train loss: 0.07816
Epoch 4, age val loss: 0.017

Epoch 2, val loss: 8.66747 -> 7.81785, train loss: 7.89387
Epoch 3, val loss: 7.81785 -> 6.57732, train loss: 7.12297
Epoch 4, val loss: 6.57732 -> 5.94922, train loss: 6.40640
Epoch 5, val loss: 8.97289, train loss: 5.76383
Epoch 6, val loss: 5.94922 -> 5.59792, train loss: 5.40051
Epoch 7, val loss: 6.14571, train loss: 5.03453
Epoch 8, val loss: 5.80466, train loss: 4.74514
Epoch 9, val loss: 5.85690, train loss: 4.41302

--------------------------------- Prune 80 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.14117, train loss: 0.15072
Epoch 0, age val loss: 0.03061, gender val loss: 0.04046, ethnicity val loss: 0.07009
Epoch 1, val loss: 0.14117 -> 0.08382, train loss: 0.10963
Epoch 1, age val loss: 0.01927, gender val loss: 0.02581, ethnicity val loss: 0.03874
Epoch 2, val loss: 0.08382 -> 0.08190, train loss: 0.09307
Epoch 2, age val loss: 0.01894, gender val loss: 0.02573, ethnicity val loss: 0.03723
Epoch 3

#### Gender

In [8]:
task = 'gender'
num_epochs = 10

for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.16809, train loss: 0.15701
Epoch 0, age val loss: 0.06767, gender val loss: 0.03984, ethnicity val loss: 0.06058
Epoch 1, val loss: 0.16809 -> 0.10571, train loss: 0.11271
Epoch 1, age val loss: 0.02388, gender val loss: 0.02941, ethnicity val loss: 0.05243
Epoch 2, val loss: 0.10571 -> 0.08599, train loss: 0.09637
Epoch 2, age val loss: 0.02035, gender val loss: 0.02666, ethnicity val loss: 0.03899
Epoch 3, val loss: 0.09771, train loss: 0.08618
Epoch 3, age val loss: 0.02326, gender val loss: 0.02655, ethnicity val loss: 0.04791
Epoch 4, val loss: 0.09429, train loss: 0.07688
Epoch 4, age val loss: 0.02480, gender val loss: 0.02427, ethnicity val loss: 0.04522
Epoch 5, val loss: 0.08599 -> 0.08455, train loss: 0.06948
Epoch 5, age val loss: 0.01644, gender val loss: 0.03134, ethnicity val loss: 0.03678
Epoch 6, val loss: 0.084

Epoch 9, val loss: 0.02152, train loss: 0.00349

--------------------------------- Prune 40 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.11467, train loss: 0.15510
Epoch 0, age val loss: 0.03341, gender val loss: 0.03067, ethnicity val loss: 0.05058
Epoch 1, val loss: 0.11467 -> 0.09981, train loss: 0.10919
Epoch 1, age val loss: 0.02618, gender val loss: 0.02697, ethnicity val loss: 0.04667
Epoch 2, val loss: 0.09981 -> 0.08642, train loss: 0.09666
Epoch 2, age val loss: 0.01979, gender val loss: 0.02685, ethnicity val loss: 0.03978
Epoch 3, val loss: 0.10317, train loss: 0.08515
Epoch 3, age val loss: 0.03079, gender val loss: 0.02996, ethnicity val loss: 0.04242
Epoch 4, val loss: 0.08642 -> 0.08416, train loss: 0.07483
Epoch 4, age val loss: 0.01970, gender val loss: 0.02700, ethnicity val loss: 0.03746
Epoch 5, val loss: 0.08416 -> 0.08195, train loss: 0.06872
Epoch 5, age val loss: 0.01835, gender val loss: 

Epoch 5, val loss: 0.01700 -> 0.01609, train loss: 0.01543
Epoch 6, val loss: 0.01634, train loss: 0.01439
Epoch 7, val loss: 0.01741, train loss: 0.01370
Epoch 8, val loss: 0.01807, train loss: 0.01257
Epoch 9, val loss: 0.01786, train loss: 0.01181

--------------------------------- Prune 80 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.09929, train loss: 0.15471
Epoch 0, age val loss: 0.02574, gender val loss: 0.03157, ethnicity val loss: 0.04199
Epoch 1, val loss: 0.10557, train loss: 0.11020
Epoch 1, age val loss: 0.03085, gender val loss: 0.03180, ethnicity val loss: 0.04292
Epoch 2, val loss: 0.10943, train loss: 0.09556
Epoch 2, age val loss: 0.03496, gender val loss: 0.03119, ethnicity val loss: 0.04329
Epoch 3, val loss: 0.09929 -> 0.07513, train loss: 0.08553
Epoch 3, age val loss: 0.01680, gender val loss: 0.02449, ethnicity val loss: 0.03385
Epoch 4, val loss: 0.07743, train loss: 0.07720
Epoch 4, age 

#### Ethnicity

In [9]:
task = 'ethnicity'
num_epochs = 10
            
for i in range(10):
    print(f'--------------------------------- Prune {i*10} % ---------------------------------')
    prune_pct = 1.0*i / 10
    scores, mean_lat, std_lat = pruned_mtl_training(task, prune_pct, num_epochs, train_loader, val_loader, val_dataset)
    print()

--------------------------------- Prune 0 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.13343, train loss: 0.15908
Epoch 0, age val loss: 0.04161, gender val loss: 0.03293, ethnicity val loss: 0.05889
Epoch 1, val loss: 0.13343 -> 0.08776, train loss: 0.11325
Epoch 1, age val loss: 0.02020, gender val loss: 0.02702, ethnicity val loss: 0.04054
Epoch 2, val loss: 0.08776 -> 0.08263, train loss: 0.09784
Epoch 2, age val loss: 0.01873, gender val loss: 0.02643, ethnicity val loss: 0.03747
Epoch 3, val loss: 0.08794, train loss: 0.08632
Epoch 3, age val loss: 0.02303, gender val loss: 0.02370, ethnicity val loss: 0.04121
Epoch 4, val loss: 0.08263 -> 0.07545, train loss: 0.07766
Epoch 4, age val loss: 0.01611, gender val loss: 0.02358, ethnicity val loss: 0.03576
Epoch 5, val loss: 0.08459, train loss: 0.06985
Epoch 5, age val loss: 0.01778, gender val loss: 0.02727, ethnicity val loss: 0.03953
Epoch 6, val loss: 0.089

Epoch 9, val loss: 0.04271, train loss: 0.00840

--------------------------------- Prune 40 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.11687, train loss: 0.15207
Epoch 0, age val loss: 0.02652, gender val loss: 0.03086, ethnicity val loss: 0.05950
Epoch 1, val loss: 0.11687 -> 0.10133, train loss: 0.10766
Epoch 1, age val loss: 0.02306, gender val loss: 0.03522, ethnicity val loss: 0.04306
Epoch 2, val loss: 0.11131, train loss: 0.09324
Epoch 2, age val loss: 0.03950, gender val loss: 0.03152, ethnicity val loss: 0.04028
Epoch 3, val loss: 0.10133 -> 0.08508, train loss: 0.08528
Epoch 3, age val loss: 0.01728, gender val loss: 0.02692, ethnicity val loss: 0.04088
Epoch 4, val loss: 0.08698, train loss: 0.07606
Epoch 4, age val loss: 0.02335, gender val loss: 0.02285, ethnicity val loss: 0.04078
Epoch 5, val loss: 0.10570, train loss: 0.06741
Epoch 5, age val loss: 0.02983, gender val loss: 0.03006, ethnicity val

Epoch 6, val loss: 0.04231 -> 0.04185, train loss: 0.03967
Epoch 7, val loss: 0.04187, train loss: 0.03749
Epoch 8, val loss: 0.04451, train loss: 0.03522
Epoch 9, val loss: 0.04185 -> 0.04167, train loss: 0.03363

--------------------------------- Prune 80 % ---------------------------------
---------------- Train Initial Model ----------------
Epoch 0, val loss: inf -> 0.10975, train loss: 0.15434
Epoch 0, age val loss: 0.02554, gender val loss: 0.03534, ethnicity val loss: 0.04887
Epoch 1, val loss: 0.10975 -> 0.10632, train loss: 0.11043
Epoch 1, age val loss: 0.02532, gender val loss: 0.03087, ethnicity val loss: 0.05013
Epoch 2, val loss: 0.10632 -> 0.08927, train loss: 0.09511
Epoch 2, age val loss: 0.02219, gender val loss: 0.02971, ethnicity val loss: 0.03737
Epoch 3, val loss: 0.08927 -> 0.08058, train loss: 0.08271
Epoch 3, age val loss: 0.01642, gender val loss: 0.02808, ethnicity val loss: 0.03607
Epoch 4, val loss: 0.08058 -> 0.07832, train loss: 0.07613
Epoch 4, age val 